## 0 - Setting CUDA VISIBLE DEVICES and importing packages

Selecting the GPUs...

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"

Importing packages...

In [ ]:
import tensorflow as tf
import sys
sys.path.insert(1, "..")

from utils import add_temporal_dim
import create_models
import numpy as np
import tensorflow.keras.layers
from notebook_utils import extract_dataset, extract_model_info, energy_estimation
from utils import COLOUR_DICTIONARY

## 2 - Notebook parameters

In [ ]:
n_timesteps=10 #Number of timesteps
dt=0.1 # [s] - Timestep size
n_test_images=10 # Number of images used for energy estimation
train_percentage=0.8 # Percentage of train dataset over the entire images
test_percentage=0.15 #Percentage of test dataset over the entire images (Note: valid_percentage=1-(train_percentage+test_percentage))
notebook_verbose=False #If true, all the notebook is executed in verbose mode

## 3 - Creating a spiking model

Loading a pretrained Spiking VGG16 model pretrained on ImageNet...

In [ ]:
model = create_models.create_spiking_vgg16_model('', input_shape=(64, 64, 3), num_classes=10)

Showing model summary...

In [ ]:
if notebook_verbose:
    model.summary()

## 4 - Extracting and formatting EuroSAT datasets

Extracting datasets.

In [ ]:
[train_ds, valid_ds, test_ds] = extract_dataset(dataset_path="../datasets/EuroSAT", train_percentage=train_percentage, test_percentage=test_percentage)

In [ ]:
train_ds =train_ds.take(n_test_images) #Limiting the number of images to n_test_images

Reshaping dataset in time domain.

In [ ]:
x_train_t = train_ds.map(add_temporal_dim(timesteps=n_timesteps), num_parallel_calls=tf.data.experimental.AUTOTUNE)

## 5 - Calculating model energy

Energy estimation for artificial model.

In [ ]:
artificial_synop_energy, artificial_neuron_energy=energy_estimation(model, x_test_t=None, spiking_model=False, device_list=['cpu','gpu', 'arm','myriad2','loihi', 'spinnaker','spinnaker2'], n_timesteps=n_timesteps, dt=dt, verbose=notebook_verbose)

Energy estimation for spiking model.

In [ ]:
spiking_synop_energy, spiking_neuron_energy=energy_estimation(model, x_test_t=x_train_t, spiking_model=True, device_list=['loihi', 'spinnaker','spinnaker2'], n_timesteps=n_timesteps, dt=dt, verbose=notebook_verbose)